In [1]:
pip install gradio torch transformers google-generativeai PyPDF2 python-pptx beautifulsoup4 langdetect pandas openpyxl python-docx pillow numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 13.3 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=423e9165f92e291bccf7b5191d224affdaa4f2b44fc79b2f063ee7690d5c2e93
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import google.generativeai as genai
import os
import requests
from PyPDF2 import PdfReader
from pptx import Presentation
from bs4 import BeautifulSoup
from langdetect import detect
import json
import re
from datetime import datetime
import time
from google.colab import userdata
import pandas as pd
import openpyxl
import docx
import base64
from PIL import Image
import io
import numpy as np

# Configuration
class Config:
    granite_model = "ibm-granite/granite-3.3-2b-instruct"
    gemini_model = "gemini-2.5-flash-image-preview"
    gemini_api_key = userdata.get('gemini_api_key')
    device = "cuda" if torch.cuda.is_available() else "cpu"

# Global models
granite_tokenizer, granite_model, gemini_model = None, None, None

# Initialize models with automatic API key from secrets
def initialize_models():
    """Initialize both Granite and Gemini models with API key from Colab secrets"""
    try:
        global granite_tokenizer, granite_model, gemini_model

        # Get API key from Colab secrets
        api_key = Config.gemini_api_key
        if not api_key:
            return "❌ Error: Gemini API key not found in Colab secrets. Please add 'gemini_api_key' to your secrets."

        # Initialize Granite model
        print(f"🔄 Loading AI models on {Config.device}...")
        granite_tokenizer = AutoTokenizer.from_pretrained(Config.granite_model, trust_remote_code=True)

        if granite_tokenizer.pad_token is None:
            granite_tokenizer.pad_token = granite_tokenizer.eos_token

        granite_model = AutoModelForCausalLM.from_pretrained(
            Config.granite_model,
            torch_dtype=torch.float16 if Config.device == "cuda" else torch.float32,
            device_map="auto" if Config.device == "cuda" else None,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )

        if Config.device == "cpu":
            granite_model = granite_model.to(Config.device)

        # Initialize Gemini
        genai.configure(api_key=api_key)
        gemini_model = genai.GenerativeModel(Config.gemini_model)

        return f"✅ Ready! Models loaded on {Config.device}. AI assistant is now active."
    except Exception as e:
        return f"❌ Setup failed: {str(e)}"

# File processing functions
def extract_text_from_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def extract_text_from_image(image):
    """Extract text from image using Gemini Vision"""
    try:
        if gemini_model is None:
            return "Gemini model not initialized"

        response = gemini_model.generate_content([
            "Extract and describe all text and visual elements from this image. Be detailed and comprehensive.",
            image
        ])
        return response.text
    except Exception as e:
        return f"Error processing image: {str(e)}"

def process_multimodal_input(message):
    """Process multimodal input (text + files/images)"""
    if isinstance(message, dict):
        text = message.get("text", "")
        files = message.get("files", [])

        content = text
        if files:
            for file in files:
                if file.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    # Process image
                    image = Image.open(file)
                    image_content = extract_text_from_image(image)
                    content += f"\n\nImage Content:\n{image_content}"
                else:
                    # Process other file types
                    file_content = process_file_content(file)
                    content += f"\n\nFile Content:\n{file_content}"

        return content
    return message

def process_file_content(file_path):
    """Process various file types"""
    try:
        if file_path.endswith('.pdf'):
            return extract_text_from_pdf(file_path)
        elif file_path.endswith('.docx'):
            doc = docx.Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        elif file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
            return df.to_string()
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
            return df.to_string()
        elif file_path.endswith('.txt'):
            with open(file_path, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            return f"Unsupported file type: {file_path}"
    except Exception as e:
        return f"Error processing file: {str(e)}"

# Streaming response generator
def stream_granite_response(prompt, max_tokens=1000):
    """Generate streaming response from Granite model with thinking process"""
    try:
        if granite_model is None or granite_tokenizer is None:
            yield "❌ Models not initialized. Please initialize first."
            return

        # Show thinking process
        yield "🤔 **Thinking:** Analyzing your request...\n\n"
        time.sleep(0.5)

        yield "🤔 **Thinking:** Processing content and preparing response...\n\n"
        time.sleep(0.5)

        yield "🤔 **Thinking:** Generating comprehensive answer...\n\n"
        time.sleep(0.5)

        # Prepare inputs
        inputs = granite_tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
            padding=True
        )
        inputs = {k: v.to(granite_model.device) for k, v in inputs.items()}

        # Generate response with streaming simulation
        with torch.no_grad():
            output_ids = granite_model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=0.7,
                do_sample=True,
                pad_token_id=granite_tokenizer.pad_token_id,
                eos_token_id=granite_tokenizer.eos_token_id,
                repetition_penalty=1.1
            )

        # Decode response
        response = granite_tokenizer.decode(
            output_ids[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )

        # Clear thinking and show response
        yield ""

        # Stream response word by word
        words = response.split()
        current_response = ""
        for i, word in enumerate(words):
            current_response += word + " "
            if i % 3 == 0:  # Update every 3 words
                yield current_response
                time.sleep(0.1)

        # Final response
        yield current_response.strip() if current_response.strip() else "I apologize, but I couldn't generate a proper response. Please try rephrasing your request."

    except Exception as e:
        yield f"❌ Error generating response: {str(e)}"

# Main chat function for gr.ChatInterface (messages-based)
def chat_with_edututor(messages):
    """
    Main chat function with multimodal support and streaming.
    Accepts a list of messages (dicts with 'role' and 'content'), builds prompt, and returns updated messages with assistant's response.
    """
    # Find last user message (possibly multimodal)
    latest_user_message = None
    for m in reversed(messages):
        if m.get("role") == "user":
            latest_user_message = m
            break
    if latest_user_message is not None:
        content = process_multimodal_input(latest_user_message.get("content", ""))
    else:
        content = ""

    # Build conversation context from previous exchanges (last 3)
    context = ""
    # Collect last 3 user/assistant pairs
    pairs = []
    user_msg = None
    for m in messages:
        if m.get("role") == "user":
            user_msg = m.get("content", "")
        elif m.get("role") == "assistant" and user_msg is not None:
            pairs.append((user_msg, m.get("content", "")))
            user_msg = None
    if pairs:
        context = "Previous conversation:\n"
        for user_msg, bot_msg in pairs[-3:]:
            context += f"User: {user_msg}\nAssistant: {bot_msg}\n"

    prompt = f"""You are Edututor, an advanced AI learning assistant. You help students learn by explaining concepts clearly, answering questions, creating study plans, and providing educational guidance.

{context}

Current request: {content}

Please provide a helpful, educational response. Use clear explanations, examples when appropriate, and structure your response well. If the user uploaded files or images, analyze and incorporate that content into your response."""

    # Use streaming generator, accumulate response
    response = ""
    for partial_response in stream_granite_response(prompt):
        response = partial_response
        yield messages + [{"role": "assistant", "content": response}]

# Create the enhanced interface
def create_edututor_interface():
    # Enhanced CSS with monospace fonts and new color scheme
    custom_css = """
    @import url('https://fonts.googleapis.com/css2?family=JetBrains+Mono:wght@300;400;500;600;700&display=swap');

    :root {
        --primary-color: #2563eb;
        --secondary-color: #1e40af;
        --accent-color: #f59e0b;
        --success-color: #10b981;
        --error-color: #ef4444;
        --warning-color: #f59e0b;
        --bg-primary: #0f172a;
        --bg-secondary: #1e293b;
        --bg-tertiary: #334155;
        --text-primary: #f8fafc;
        --text-secondary: #cbd5e1;
        --border-color: #475569;
    }

    * {
        font-family: 'JetBrains Mono', 'Consolas', 'Monaco', 'Courier New', monospace !important;
    }

    .gradio-container {
        background: linear-gradient(135deg, var(--bg-primary), var(--bg-secondary)) !important;
        color: var(--text-primary) !important;
        min-height: 100vh !important;
    }

    .sidebar {
        background: rgba(30, 41, 59, 0.9) !important;
        backdrop-filter: blur(20px) !important;
        border-right: 1px solid var(--border-color) !important;
        border-radius: 0 20px 20px 0 !important;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3) !important;
        padding: 24px !important;
        min-height: 100vh !important;
    }

    .main-content {
        background: rgba(15, 23, 42, 0.8) !important;
        backdrop-filter: blur(20px) !important;
        margin-top: 10px !important;
        border-radius: 25px !important;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.2) !important;
        border: 1px solid var(--border-color) !important;
        padding: 30px !important;
    }

    .edututor-title {
        font-size: 42px !important;
        font-weight: 700 !important;
        color: var(--accent-color) !important;
        margin-bottom: 12px !important;
        text-shadow: 0 0 20px rgba(245, 158, 11, 0.3) !important;
        letter-spacing: -0.02em !important;
    }

    .subtitle {
        color: var(--text-secondary) !important;
        font-size: 16px !important;
        font-weight: 400 !important;
        margin-bottom: 24px !important;
        opacity: 0.9 !important;
    }

    .nav-button {
        width: 100% !important;
        text-align: left !important;
        padding: 14px 18px !important;
        margin: 6px 0 !important;
        border: none !important;
        background: rgba(51, 65, 85, 0.5) !important;
        border-radius: 12px !important;
        color: var(--text-secondary) !important;
        transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
        border: 1px solid transparent !important;
        font-weight: 500 !important;
    }

    .nav-button:hover {
        background: rgba(37, 99, 235, 0.2) !important;
        border-color: var(--primary-color) !important;
        color: var(--text-primary) !important;
        transform: translateX(4px) !important;
    }

    .nav-button.active {
        background: linear-gradient(135deg, var(--primary-color), var(--secondary-color)) !important;
        color: white !important;
        border-color: var(--primary-color) !important;
        box-shadow: 0 4px 16px rgba(37, 99, 235, 0.3) !important;
    }

    .chat-interface {
        background: rgba(30, 41, 59, 0.6) !important;
        border-radius: 16px !important;
        border: 1px solid var(--border-color) !important;
        backdrop-filter: blur(10px) !important;
    }

    .message-user {
        background: linear-gradient(135deg, var(--primary-color), var(--secondary-color)) !important;
        color: white !important;
        border-radius: 18px 18px 4px 18px !important;
        padding: 12px 16px !important;
        margin: 8px !important;
        border: none !important;
    }

    .message-bot {
        background: rgba(51, 65, 85, 0.8) !important;
        color: var(--text-primary) !important;
        border-radius: 18px 18px 18px 4px !important;
        padding: 12px 16px !important;
        margin: 8px !important;
        border: 1px solid var(--border-color) !important;
    }

    .input-area {
        background: rgba(30, 41, 59, 0.8) !important;
        border: 1px solid var(--border-color) !important;
        border-radius: 20px !important;
        color: var(--text-primary) !important;
    }

    .input-area:focus {
        border-color: var(--accent-color) !important;
        box-shadow: 0 0 0 3px rgba(245, 158, 11, 0.2) !important;
    }

    .submit-button {
        background: linear-gradient(135deg, var(--accent-color), #d97706) !important;
        border: none !important;
        border-radius: 12px !important;
        color: white !important;
        font-weight: 600 !important;
        padding: 10px 20px !important;
        transition: all 0.3s ease !important;
    }

    .submit-button:hover {
        transform: translateY(-2px) !important;
        box-shadow: 0 8px 25px rgba(245, 158, 11, 0.3) !important;
    }

    .status-indicator {
        background: rgba(16, 185, 129, 0.2) !important;
        border: 1px solid var(--success-color) !important;
        color: var(--success-color) !important;
        padding: 12px 18px !important;
        border-radius: 12px !important;
        margin: 12px 0 !important;
        backdrop-filter: blur(10px) !important;
    }

    .thinking-indicator {
        background: rgba(245, 158, 11, 0.2) !important;
        border: 1px solid var(--accent-color) !important;
        color: var(--accent-color) !important;
        padding: 8px 12px !important;
        border-radius: 8px !important;
        font-style: italic !important;
        animation: pulse 2s infinite !important;
    }

    @keyframes pulse {
        0%, 100% { opacity: 1; }
        50% { opacity: 0.7; }
    }

    .feedback-buttons {
        display: flex !important;
        gap: 8px !important;
        margin-top: 8px !important;
    }

    .feedback-btn {
        background: rgba(51, 65, 85, 0.5) !important;
        border: 1px solid var(--border-color) !important;
        border-radius: 8px !important;
        padding: 6px 12px !important;
        color: var(--text-secondary) !important;
        transition: all 0.3s ease !important;
    }

    .feedback-btn:hover {
        background: rgba(37, 99, 235, 0.2) !important;
        border-color: var(--primary-color) !important;
        color: var(--text-primary) !important;
    }

    .history-panel {
        background: rgba(30, 41, 59, 0.8) !important;
        border: 1px solid var(--border-color) !important;
        border-radius: 16px !important;
        backdrop-filter: blur(10px) !important;
    }

    .file-upload-area {
        border: 2px dashed var(--border-color) !important;
        border-radius: 16px !important;
        background: rgba(51, 65, 85, 0.3) !important;
        padding: 24px !important;
        text-align: center !important;
        transition: all 0.3s ease !important;
    }

    .file-upload-area:hover {
        border-color: var(--accent-color) !important;
        background: rgba(245, 158, 11, 0.1) !important;
    }

    /* Scrollbar styling */
    ::-webkit-scrollbar {
        width: 9.8px !important;

    }

    ::-webkit-scrollbar-track {
        background: rgba(30, 41, 59, 0.3) !important;
        border-radius: 4px !important;
        backdrop-filter:blur(10px);
    }

    ::-webkit-scrollbar-thumb {
        background: var(--border-color) !important;
        border-radius: 4px !important;
        -webkit-backdrop-filter:blur(10px);
    }

    ::-webkit-scrollbar-thumb:hover {
        background: var(--accent-color) !important;
    }
    """

    # Create the main interface using gr.ChatInterface
    with gr.Blocks(css=custom_css, title="Guru - Advanced AI Learning Assistant", theme=gr.themes.Base()) as app:

        # Initialize models on startup
        gr.HTML("<script>window.onload = function() { console.log('our guru is loaded'); }</script>")

        with gr.Row():
            # Sidebar
            with gr.Column(scale=1, elem_classes=["sidebar"]):
                gr.HTML('<h1 class="edututor-title">Guru</h1>')
                gr.HTML('<p class="subtitle">Advanced AI Learning Assistant</p>')

                # Status indicator
                with gr.Group():
                    status_display = gr.HTML('<div class="status-indicator">Initializing...</div>')
                    init_btn = gr.Button("🚀Initialize System", variant="primary", size="sm")

                gr.HTML("<hr style='margin: 24px 0; border-color: var(--border-color); opacity: 0.3;'>")

                # Navigation
                gr.HTML('<div style="font-size: 15px; color: var(--text-secondary); margin-bottom: 12px; font-weight: 600; backdrop-filter:blur(10px)">NAVIGATION</div>')
                gr.Button(" Home", elem_classes=["nav-button", "active"])
                gr.Button(" New Chat", elem_classes=["nav-button"])
                gr.Button(" My Tasks", elem_classes=["nav-button"])
                gr.Button(" My Meetings", elem_classes=["nav-button"])
                gr.Button(" Saved Files", elem_classes=["nav-button"])
                gr.Button(" Shared Items", elem_classes=["nav-button"])

                gr.HTML("<hr style='margin: 24px 0; border-color: var(--border-color); opacity: 0.3;'>")

                # Quick Actions
                gr.HTML('<div style="font-size: 14px; color: var(--text-secondary); margin-bottom: 12px; font-weight: 600;">TODAY\'S ACTIVITIES</div>')
                gr.Button(" Research Request", elem_classes=["nav-button"])
                gr.Button("Meeting Summary", elem_classes=["nav-button"])
                gr.Button("Task Prioritization", elem_classes=["nav-button"])
                gr.Button(" Study Planning", elem_classes=["nav-button"])

            # Main Chat Interface
            with gr.Column(scale=3, elem_classes=["main-content"]):

                # Advanced ChatInterface with all features
                chat_interface = gr.ChatInterface(
                    fn=chat_with_edututor,
                    title="🎓 Edututor Chat",
                    description="Upload files, images, or ask questions. I can help with learning, research, and academic tasks!",
                    multimodal=True,  # Enable file/image uploads
                    save_history=True,  # Enable persistent chat history
                    flagging_mode="manual",  # Enable user feedback (like/dislike)
                    flagging_options=["👍 Helpful", "👎 Not Helpful", "🔄 Needs Improvement", "⭐ Excellent"],
                    flagging_dir="./chat_feedback",
                    show_progress="full",
                    fill_height=True,
                    type="messages",
                )

        # Initialize system function
        def initialize_system():
            status = initialize_models()
            if "✅" in status:
                return f'<div class="status-indicator">{status}</div>'
            else:
                return f'<div style="background: rgba(239, 68, 68, 0.2); border: 1px solid var(--error-color); color: var(--error-color); padding: 12px 18px; border-radius: 12px;">{status}</div>'

        # Connect initialization
        init_btn.click(
            initialize_system,
            outputs=[status_display]
        )

        # Auto-initialize on load
        app.load(
            initialize_system,
            outputs=[status_display]
        )

    return app

# Voice conversation feature (optional enhancement)
def create_voice_interface():
    """Create voice conversation interface using speech recognition and TTS"""

    def process_voice_input(audio):
        # This would integrate with speech recognition
        # For now, return placeholder
        return "Voice input received and processed"

    def text_to_speech(text):
        # This would integrate with TTS service
        # Return audio file path
        return text

    with gr.Blocks() as voice_app:
        gr.HTML("<h2>Interact with Guru</h2>")

        with gr.Row():
            audio_input = gr.Audio(source="microphone", type="filepath")
            audio_output = gr.Audio()

        voice_btn = gr.Button(" Start Voice Conversation")

        voice_btn.click(
            process_voice_input,
            inputs=[audio_input],
            outputs=[audio_output]
        )

    return voice_app

# Launch the application
if __name__ == "__main__":
    app = create_edututor_interface()
    app.launch(
        debug=True,
        share=True,
        server_name="0.0.0.0",
        show_error=True,
        inbrowser=True,
        max_file_size="50mb"  # Allow larger file uploads
    )

/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1052: UserWarning: Expected 1 arguments for function <function chat_with_edututor at 0x7a5e35b1b740>, received 2.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1060: UserWarning: Expected maximum 1 arguments for function <function chat_with_edututor at 0x7a5e35b1b740>, received 2.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1052: UserWarning: Expected 1 arguments for function <function chat_with_edututor at 0x7a5e35b1b7e0>, received 2.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1060: UserWarning: Expected maximum 1 arguments for function <function chat_with_edututor at 0x7a5e35b1b7e0>, received 2.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f9d9b1f79d89ff62ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 Loading AI models on cpu...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🔄 Loading AI models on cpu...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]